### Loading Libraries


In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.llms import Ollama




In [13]:
loader = PyPDFDirectoryLoader("papers")
documents = loader.load()

KeyboardInterrupt: 

In [ ]:
len(documents)

1219

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [ ]:
len(chunks)

3841

In [2]:
model = SentenceTransformerEmbeddings(model_name="mukaj/fin-mpnet-base",model_kwargs={'device': 'cuda:0'})

In [3]:
#db = Chroma.from_documents(documents=chunks,embedding=model,persist_directory="./chroma_db")
db = Chroma(persist_directory="./chroma_db",embedding_function=model)

In [4]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager


llm = Ollama(model='mistral-7b-instruct-v0.2',callbacks=[StreamingStdOutCallbackHandler()])

## Retreval + Generation

In [5]:
retriever = db.as_retriever(search_kwargs={"k":5})

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import langchain 
#langchain.debug = False
from langchain_core.output_parsers import StrOutputParser
template = """<s>[INST] Context information is below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {input}
Answer: [/INST]"
"""
prompt = ChatPromptTemplate.from_template(template)

In [7]:
rag_chain = (
    {"context":retriever,"input":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
#response = rag_chain.invoke("Give me the Cash From Investing Activities in 2023")
#print(response[response.find('[/INST]')+7:].replace(r'\n', '+ "\n" +'))

In [9]:
query = "What is the Enterprise value to EBITDA ratio for 2023?"
for chunks in rag_chain.stream(query):
    print(chunks, end='')


 To To calculate calculate the the Enterprise Enterprise Value Value ( (EVEV)) to to E Earnarningsings Before Before Interest Interest,, Tax Taxeses,, Dep Deprecireciationation,, and and Am Amortortizationization ( (EEBITBITDADA)) ratio ratio,, we we need need additional additional information information such such as as the the EV EV and and E EBITBITDADA figures figures for for  22002233.. The The context context provided provided includes includes earnings earnings per per share share data data but but does does not not explicitly explicitly state state the the EV EV or or E EBITBITDADA figures figures for for that that year year.. Therefore Therefore,, it it is is impossible impossible to to definit definitivelyively answer answer the the query query with with the the given given context context alone alone..



HoweverHowever,, if if you you have have access access to to additional additional financial financial statements statements or or reports reports from from Y YTLTL Corpora